In [37]:
import requests
from requests_futures.sessions import FuturesSession
import re
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
from urllib.parse import urldefrag, urlparse, urljoin, urlsplit, urlunsplit
session = FuturesSession()

In [74]:
def get_response(url):
    try:
        res = requests.get(url)
        if (res.ok):
            print("Status %s" % (res.status_code))
            return res
        else:
            res.raise_for_status()
            
    except Exception as e:
        print(url, e)


def parse_content(res_object):
    if (res_object):
        parse_only = SoupStrainer(["a", "link", "meta", "title"])
        s = BeautifulSoup(res_object.content, "lxml", parse_only=parse_only)
        return s
    else:
        return False;

def get_urls(parsed_html, element=""):
    if (parsed_html):
        pattern = re.compile(u"(rss)|([./\+]xml$)|(xml;.*)", flags=re.I)
        reject_pattern = re.compile(u"(\/comments\/)", flags=re.I)
        links = (alink.get("href")
                 for alink in parsed_html.findAll(element)
                 if re.findall(pattern, str(alink)) 
                 and not re.findall(reject_pattern, str(alink)))
        return links
    else:
        return []

def parse_meta_attrs(parsed_html):
    if (parsed_html):
        pattern = re.compile(u"twitter:|fb:|description|og:|keywords", flags=re.I)
        try:
            metas = {attrs["property"]: attrs["content"] for attrs in 
                     [meta.attrs for meta in parsed_html.findAll("meta") 
                      if "property=" in str(meta) and re.findall(pattern, str(meta))]}
            named_metas = {attrs["name"]: attrs["content"] for attrs in 
                     [meta.attrs for meta in parsed_html.findAll("meta") 
                      if "name=" in str(meta) and re.findall(pattern, str(meta))]}
            metas.update(named_metas)
            metas["title"] = parsed_html.find("title").string
            return metas
        except Exception as e:
            print(e)
            return None
    else:
        return None

    
def check_feed_response(url):
    if (url):
        try:
            res = requests.get(url)
            if (res.ok):
                return res.headers["content-type"]
            else:
                res.raise_for_status()
        except Exception as e:
            print(e)
            return res.status_code
    else:
        return None
    
def get_site_info(url):
    site_info = {"url": url}
    print("%s: \nGetting response..." % (url))
    res = get_response(url)
    print("Parsing content...")
    html = parse_content(res)
    site_info["rss_links"] = list(set(get_urls(html, "link")))
    site_info["rss_links"] = [urljoin(url, link, allow_fragments=False) for link in site_info["rss_links"]]
    site_info["rss_leads"] = set(get_urls(html, "a"))
    site_info["rss_leads"] = (urljoin(url, link, allow_fragments=False) for link in site_info["rss_leads"])
    site_info["rss_leads"] = list(set(link for link in site_info["rss_leads"] if link not in site_info["rss_links"]))
    print("found %d RSS feed(s) and %d link(s) to possible feeds" % 
          (len(site_info["rss_links"]), len(site_info["rss_leads"])))
    site_info["meta"] = parse_meta_attrs(html)
    
    return site_info

In [75]:
def get_domain_link(url):
    scheme, netloc = urlsplit(url)[:2]
    link = urlunsplit((scheme, netloc,"", "", ""))
    return link

res = requests.get("https://news.google.com/news?cf=all&pz=1&ned=uk&siidp=d99c9ccef53edd7975a8b314decb9a979877&ict=ln")
s = BeautifulSoup(res.content,"lxml")
links = [a.get("href") for a in s.findAll("a") if not (re.findall(u"google\.com", str(a.get("href"))) or not str(a.get("href")).startswith("http"))]
links = pd.Series(([get_domain_link(link) for link in links]))
links = links.drop_duplicates()
links = links.loc[links != b'',]

In [33]:
links.to_csv("glinks.csv", sep="\t", encoding="UTF-8")

In [111]:
dict_list = (get_site_info(url) for url in links)

In [112]:
%time dicts = list(dict_list)

https://www.youtube.com: 
Getting response...
Status 200
Parsing content...
found 0 RSS feed(s) and 0 link(s) to possible feeds
http://www.mirror.co.uk: 
Getting response...
Status 200
Parsing content...
found 1 RSS feed(s) and 2 link(s) to possible feeds
http://www.newstatesman.com: 
Getting response...
Status 200
Parsing content...
found 0 RSS feed(s) and 1 link(s) to possible feeds
https://www.theguardian.com: 
Getting response...
Status 200
Parsing content...
found 1 RSS feed(s) and 0 link(s) to possible feeds
http://www.walesonline.co.uk: 
Getting response...
Status 200
Parsing content...
found 1 RSS feed(s) and 4 link(s) to possible feeds
http://www.dailymail.co.uk: 
Getting response...
Status 200
Parsing content...
found 2 RSS feed(s) and 1 link(s) to possible feeds
http://www.bbc.co.uk: 
Getting response...
Status 200
Parsing content...
found 0 RSS feed(s) and 0 link(s) to possible feeds
http://www.ft.com: 
Getting response...
Status 200
Parsing content...
found 1 RSS feed(s) a

In [126]:
from itertools import chain
rss_links = filter(None, map(lambda x: x["rss_links"], dicts))

In [127]:
list_or_rss_links = list(chain.from_iterable(rss_links))

In [128]:
with open("rss_links.csv", mode="w", encoding="UTF-8") as f:
    for line in list_or_rss_links:
        f.write("%s\n" % (line))

In [94]:
leads = (get_site_info(x) for x in dicts[0]["rss_leads"])
list(leads)

http://uk.reuters.com/tools/rss: 
Getting response...
Status 200
Parsing content...
found 1 RSS feed(s) and 38 link(s) to possible feeds


[{'meta': {'description': 'Reuters.co.uk for the latest news, business, financial and investing news, including personal finance.',
   'keywords': 'Breaking News,World News,Video news,Investing,financial information, Business News,Rueters,Roiters',
   'og:description': 'Reuters.co.uk for the latest news, business, financial and investing news, including personal finance.',
   'og:locale': 'en_GB',
   'og:site_name': 'Reuters UK',
   'og:title': 'Latest News | Reuters.co.uk',
   'og:type': 'section',
   'og:url': 'http://uk.reuters.com/tools/rss',
   'title': 'Latest News | Reuters.co.uk',
   'twitter:card': 'summary_large_image',
   'twitter:creator': '@ReutersUK',
   'twitter:description': 'Reuters.co.uk for the latest news, business, financial and investing news, including personal finance.',
   'twitter:site': '@ReutersUK'},
  'rss_leads': ['http://add.my.yahoo.com/rss?url=http://feeds.reuters.com/reuters/UKVideoRoughCuts',
   'http://add.my.yahoo.com/rss?url=http://feeds.reuters.co

In [ ]:
df = urls[10:20].copy()
df["content"] = df["url"].apply(get_response).apply(parse_content)
df["links"] = df["content"].apply(get_urls, element = "link").apply(list)
df["leads"] = df["content"].apply(get_urls, element = "a").apply(list)

In [ ]:
f = df.apply(lambda x: pd.Series(x['links']),axis=1).stack().reset_index(level=1, drop=True)
f.name = 'links'
df = df.drop('links', axis=1).join(f)
f = df.apply(lambda x: pd.Series(x['leads']),axis=1).stack().reset_index(level=1, drop=True)
f.name = 'leads'
df = df.drop('leads', axis=1).join(f)

In [ ]:
df = df.drop_duplicates()
df = df.fillna("")
df["url"] = df["url"].str.rstrip("/")
df["links"] = df["links"].str.rstrip("/")
df["leads"] = df["leads"].str.rstrip("/")

In [ ]:
df["links"] = df.apply(lambda x: urljoin(x["url"], str(x["links"])), axis=1)
df["leads"] = df.apply(lambda x: urljoin(x["url"], str(x["leads"])), axis=1)

In [ ]:
df["link_type"] = df["links"].apply(check_feed_response) 

In [ ]:
df

In [ ]:
df.to_csv("test.csv", sep="\t", encoding="UTF-8")

In [ ]:
df["test"] = df["links"] != df["leads"]

In [ ]:
df["og"] = df["content"].apply(parse_opengraph_attrs)

In [ ]:
df

In [129]:
dicts

[{'meta': {'description': 'Auf YouTube findest du großartige Videos und erstklassige Musik. Außerdem kannst du eigene Inhalte hochladen und mit Freunden oder mit der ganzen Welt teilen.',
   'fb:app_id': '87741124305',
   'keywords': 'Video, E-Mail, Kamerahandy, Videohandy, kostenlos, hochladen',
   'og:image': '//s.ytimg.com/yts/img/yt_1200-vfl4C3T0K.png',
   'title': 'YouTube'},
  'rss_leads': [],
  'rss_links': [],
  'url': 'https://www.youtube.com'},
 {'meta': {'description': 'All the latest news, sport and celebrity gossip at Mirror.co.uk. Get all the big headlines, pictures, analysis, opinion and video on the stories that matter to you.',
   'fb:app_id': '123792601022133',
   'fb:page_id': '6149699161',
   'og:language': 'en',
   'og:site_name': 'mirror',
   'og:type': 'website',
   'og:url': 'http://www.mirror.co.uk/',
   'title': 'Mirror Online: The intelligent tabloid. #madeuthink'},
  'rss_leads': ['http://www.mirror.co.uk/tv/tv-news/love-islands-malin-andersson-feels-8287247

In [134]:
ddf = pd.read_json(json.dumps(dicts),)

In [136]:
ddf

In [27]:
def parse_meta_attrs(parsed_html):
    if (parsed_html):
        pattern = re.compile(u"twitter:|fb:|description|og:|keywords", flags=re.I)
        try:
            metas = {attrs["property"]: attrs["content"] for attrs in 
                     [meta.attrs for meta in parsed_html.findAll("meta") 
                      if "property=" in str(meta) and re.findall(pattern, str(meta))]}
            named_metas = {attrs["name"]: attrs["content"] for attrs in 
                     [meta.attrs for meta in parsed_html.findAll("meta") 
                      if "name=" in str(meta) and re.findall(pattern, str(meta))]}
            metas.update(named_metas)
            metas["title"] = parsed_html.find("title").string
            return metas
        except Exception as e:
            print(e)
            return None
    else:
        return None

In [28]:
res = get_response("http://www.tutorialspoint.com/")
html = parse_content(res)
parse_meta_attrs(html)

Status 200


{'Description': 'Free tutorials and reference manuals with examples for Computer Security, Extreme Programming, Behavior Driven Development, Conversion Rate Optimization, SAP Basis, Business Law, Kanban, Erlang, SAP Payroll, ASP.Net WP, PouchDB, MFC, SAP HR, Yii, Clojure, Aurelia, CoffeeScript, CPanel, Laravel, SAP Webi, MS SQL Server, pinterest, Indian Polity, Windows10, Teradata',
 'fb:app_id': '471319149685276',
 'og:locale': 'en_US',
 'og:site_name': 'www.tutorialspoint.com',
 'og:type': 'website',
 'title': 'Tutorials for Computer Security, Extreme Programming, Behavior Driven Development, Conversion Rate Optimization, SAP Basis, Business Law, Kanban, Erlang, SAP Payroll, ASP.Net WP, PouchDB, MFC, SAP HR, Yii, Clojure, Aurelia, CoffeeScript, CPanel, Laravel, SAP Webi, MS SQL Server, pinterest, Indian Polity, Teradata, Windows10'}